In [2]:
# https://medium.com/@apoddar573/making-your-own-custom-environment-in-gym-c3b65ff8cdaa

from livelossplot.keras import PlotLossesCallback
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np
#import tensorflow as tf
import random
import csv
import math
import os
import copy
import keras
import string
import pickle
import re
import gym
import gym_foo

from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam, SGD, Adadelta

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
import rl.callbacks

from keras.callbacks import EarlyStopping

# https://github.com/keras-rl/keras-rl/blob/master/examples/dqn_cartpole.py
ENV_NAME = 'foo-v0-7'
env = gym.make('foo-v0')
np.random.seed(123)
nb_actions = env.action_space.n

# https://qiita.com/inoory/items/e63ade6f21766c7c2393
class EpisodeLogger(rl.callbacks.Callback):
    def __init__(self):
        self.observations = {}
        self.rewards = {}
        self.actions = {}

    def on_episode_begin(self, episode, logs):
        self.observations[episode] = []
        self.rewards[episode] = []
        self.actions[episode] = []

    def on_step_end(self, step, logs):
        episode = logs['episode']
        self.observations[episode].append(logs['observation'])
        self.rewards[episode].append(logs['reward'])
        self.actions[episode].append(logs['action'])

cb_ep = EpisodeLogger()

#early_stop = EarlyStopping(patience=69) # epochs stagnation before termination

input_shape = (46, 500, 1)
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(100, kernel_size=(12,12),activation='relu', input_shape=input_shape))
model.add(keras.layers.Conv2D(50, (6, 6), padding="SAME", activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.MaxPooling2D(pool_size=(3, 3)))
model.add(keras.layers.Conv2D(20, (5, 5), padding="SAME", activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='relu'))
model.add(keras.layers.Dropout(0.8))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(61))
model.summary()

model.compile(loss="mean_squared_error",
    #optimizer=tf.keras.optimizers.Adadelta(lr=1.0),
    #optimizer=keras.optimizers.SGD(lr=0.00052),
    optimizer=keras.optimizers.Adam(0.00001),
    metrics=['accuracy'])

memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy(0.5)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=1000,
               target_model_update=1e-2, policy=policy, enable_double_dqn=True)

dqn.compile(Adam(0.0001), metrics=['mae'])
#dqn.load_weights("/home/englab5510/PycharmProjects/mmWave1/reinforcement_data/trial3/cnn_models/model_1.h5")
dqn.load_weights("foo-vo-7/dqn_foo-v0-7_weights.h5f")
#dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
#history = dqn.fit(env, nb_steps=500000, visualize=False, verbose=2, callbacks=[cb_ep]) ###

# After training is done, we save the final weights.
#dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True) ###

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=862, visualize=False)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 35, 489, 100)      14500     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 35, 489, 50)       180050    
_________________________________________________________________
dropout_4 (Dropout)          (None, 35, 489, 50)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 163, 50)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 163, 20)       25020     
_________________________________________________________________
dropout_5 (Dropout)          (None, 11, 163, 20)       0         
_________________________________________________________________
dense_3 (Dense)              (None, 11, 163, 2)       

accuracy:  2.8333333333333335
Episode 79: reward: 5.000, steps: 5
79 (array([22]),) False
accuracy:  3.0500000000000003
Episode 80: reward: 4.745, steps: 5
80 (array([48]),) False
accuracy:  1.916666666666667
Episode 81: reward: 3.758, steps: 5
81 (array([22]),) False
accuracy:  5.0
Episode 82: reward: 5.000, steps: 5
82 (array([22]),) False
accuracy:  2.75
Episode 83: reward: 5.000, steps: 5
83 (array([22]),) False
accuracy:  2.75
Episode 84: reward: 5.000, steps: 5
84 (array([22]),) False
accuracy:  2.75
Episode 85: reward: 5.000, steps: 5
85 (array([48]),) False
accuracy:  1.0833333333333335
Episode 86: reward: 4.583, steps: 5
86 (array([22]),) False
accuracy:  3.833333333333333
Episode 87: reward: 5.000, steps: 5
87 (array([3]),) False
accuracy:  1.9166666666666667
Episode 88: reward: 0.925, steps: 5
88 (array([22]),) False
accuracy:  4.75
Episode 89: reward: 5.000, steps: 5
89 (array([4]),) False
accuracy:  4.0
Episode 90: reward: 4.992, steps: 5
90 (array([22]),) False
accuracy: 

accuracy:  3.5
Episode 176: reward: 1.239, steps: 5
177 (array([38]),) False
accuracy:  2.4166666666666665
Episode 177: reward: 3.764, steps: 5
178 (array([22]),) False
accuracy:  2.716666666666667
Episode 178: reward: 4.720, steps: 5
179 (array([22]),) False
accuracy:  1.9166666666666667
Episode 179: reward: 1.823, steps: 5
180 (array([48]),) False
accuracy:  0.7833333333333334
Episode 180: reward: 0.879, steps: 5
181 (array([22]),) False
accuracy:  2.2666666666666666
Episode 181: reward: 0.850, steps: 5
182 (array([22]),) False
accuracy:  0.55
Episode 182: reward: 1.679, steps: 5
183 (array([22]),) False
accuracy:  1.9833333333333334
Episode 183: reward: 1.051, steps: 5
184 (array([22]),) False
accuracy:  1.6833333333333331
Episode 184: reward: 1.326, steps: 5
185 (array([22]),) False
accuracy:  1.7166666666666666
Episode 185: reward: 1.401, steps: 5
186 (array([16]),) False
accuracy:  1.5333333333333334
Episode 186: reward: 3.376, steps: 5
187 (array([12]),) False
accuracy:  4.33333

accuracy:  0.26666666666666666
Episode 275: reward: 5.000, steps: 5
276 (array([54]),) False
accuracy:  1.8833333333333335
Episode 276: reward: 4.395, steps: 5
277 (array([27]),) False
accuracy:  0.5833333333333334
Episode 277: reward: 3.016, steps: 5
278 (array([22]),) False
accuracy:  2.0166666666666666
Episode 278: reward: 5.000, steps: 5
279 (array([48]),) False
accuracy:  2.5833333333333335
Episode 279: reward: 3.200, steps: 5
280 (array([22]),) False
accuracy:  3.4666666666666663
Episode 280: reward: 4.863, steps: 5
281 (array([18]),) False
accuracy:  2.7333333333333334
Episode 281: reward: 4.539, steps: 5
282 (array([22]),) False
accuracy:  2.25
Episode 282: reward: 5.000, steps: 5
283 (array([36]),) False
accuracy:  2.683333333333333
Episode 283: reward: 3.293, steps: 5
284 (array([22]),) False
accuracy:  1.6
Episode 284: reward: 4.057, steps: 5
285 (array([54]),) False
accuracy:  1.0833333333333333
Episode 285: reward: 0.840, steps: 5
286 (array([48]),) False
accuracy:  3.1166

accuracy:  3.95
Episode 370: reward: 5.000, steps: 5
371 (array([48]),) False
accuracy:  2.8666666666666663
Episode 371: reward: 4.077, steps: 5
372 (array([22]),) False
accuracy:  5.0
Episode 372: reward: 5.000, steps: 5
373 (array([22]),) False
accuracy:  3.066666666666667
Episode 373: reward: 5.000, steps: 5
374 (array([22]),) False
accuracy:  3.5166666666666666
Episode 374: reward: 5.000, steps: 5
375 (array([22]),) False
accuracy:  2.7333333333333334
Episode 375: reward: 5.000, steps: 5
376 (array([48]),) False
accuracy:  1.2333333333333334
Episode 376: reward: 4.922, steps: 5
377 (array([22]),) False
accuracy:  1.5833333333333333
Episode 377: reward: 4.398, steps: 5
378 (array([22]),) False
accuracy:  1.6666666666666665
Episode 378: reward: 4.290, steps: 5
379 (array([22]),) False
accuracy:  1.65
Episode 379: reward: 4.278, steps: 5
380 (array([22]),) False
accuracy:  3.6833333333333336
Episode 380: reward: 5.000, steps: 5
381 (array([22]),) False
accuracy:  2.5166666666666666
Ep

467 (array([12]),) False
accuracy:  1.7
Episode 467: reward: 4.171, steps: 5
468 (array([22]),) False
accuracy:  1.3000000000000003
Episode 468: reward: 3.565, steps: 5
469 (array([22]),) False
accuracy:  3.4
Episode 469: reward: 2.880, steps: 5
470 (array([22]),) False
accuracy:  2.6500000000000004
Episode 470: reward: 4.071, steps: 5
471 (array([38]),) False
accuracy:  1.7000000000000002
Episode 471: reward: 3.797, steps: 5
472 (array([22]),) False
accuracy:  3.566666666666667
Episode 472: reward: 4.241, steps: 5
473 (array([22]),) False
accuracy:  1.5833333333333335
Episode 473: reward: 3.965, steps: 5
474 (array([22]),) False
accuracy:  3.033333333333333
Episode 474: reward: 4.647, steps: 5
475 (array([22]),) False
accuracy:  2.8833333333333333
Episode 475: reward: 4.514, steps: 5
476 (array([48]),) False
accuracy:  2.6833333333333336
Episode 476: reward: 4.618, steps: 5
477 (array([22]),) False
accuracy:  3.833333333333333
Episode 477: reward: 4.449, steps: 5
478 (array([22]),) Fa

accuracy:  2.716666666666667
Episode 560: reward: 4.519, steps: 5
561 (array([22]),) False
accuracy:  2.566666666666667
Episode 561: reward: 4.049, steps: 5
562 (array([22]),) False
accuracy:  0.9166666666666666
Episode 562: reward: 4.095, steps: 5
563 (array([22]),) False
accuracy:  0.95
Episode 563: reward: 4.739, steps: 5
564 (array([22]),) False
accuracy:  4.2
Episode 564: reward: 4.571, steps: 5
565 (array([12]),) False
accuracy:  2.2666666666666666
Episode 565: reward: 4.928, steps: 5
566 (array([54]),) False
accuracy:  2.0666666666666664
Episode 566: reward: 2.160, steps: 5
567 (array([22]),) False
accuracy:  2.716666666666667
Episode 567: reward: 4.320, steps: 5
568 (array([22]),) False
accuracy:  2.85
Episode 568: reward: 4.837, steps: 5
569 (array([22]),) False
accuracy:  2.4333333333333336
Episode 569: reward: 4.728, steps: 5
570 (array([22]),) False
accuracy:  3.333333333333333
Episode 570: reward: 4.794, steps: 5
571 (array([22]),) False
accuracy:  2.6833333333333336
Episo

Episode 659: reward: 4.276, steps: 5
660 (array([8]),) False
accuracy:  1.65
Episode 660: reward: 4.223, steps: 5
661 (array([22]),) False
accuracy:  1.3666666666666667
Episode 661: reward: 4.668, steps: 5
662 (array([12]),) False
accuracy:  2.8833333333333333
Episode 662: reward: 5.000, steps: 5
663 (array([58]),) False
accuracy:  2.433333333333333
Episode 663: reward: 1.978, steps: 5
664 (array([44]),) False
accuracy:  2.5
Episode 664: reward: 1.219, steps: 5
665 (array([22]),) False
accuracy:  4.833333333333333
Episode 665: reward: 4.377, steps: 5
666 (array([22]),) False
accuracy:  0.25
Episode 666: reward: 4.911, steps: 5
667 (array([22]),) False
accuracy:  1.7166666666666666
Episode 667: reward: 4.884, steps: 5
668 (array([22]),) False
accuracy:  4.166666666666666
Episode 668: reward: 3.972, steps: 5
669 (array([15]),) False
accuracy:  1.8666666666666667
Episode 669: reward: 4.388, steps: 5
670 (array([58]),) False
accuracy:  3.45
Episode 670: reward: 4.638, steps: 5
671 (array([

accuracy:  0.7666666666666666
Episode 759: reward: 4.682, steps: 5
760 (array([48]),) False
accuracy:  2.483333333333333
Episode 760: reward: 4.229, steps: 5
761 (array([3]),) False
accuracy:  2.166666666666667
Episode 761: reward: 5.000, steps: 5
762 (array([22]),) False
accuracy:  2.4833333333333334
Episode 762: reward: 4.832, steps: 5
763 (array([22]),) False
accuracy:  2.8833333333333333
Episode 763: reward: 4.142, steps: 5
764 (array([48]),) False
accuracy:  2.5833333333333335
Episode 764: reward: 4.487, steps: 5
765 (array([48]),) False
accuracy:  2.083333333333333
Episode 765: reward: 3.586, steps: 5
766 (array([48]),) False
accuracy:  1.7333333333333332
Episode 766: reward: 4.762, steps: 5
767 (array([48]),) False
accuracy:  2.2666666666666666
Episode 767: reward: 3.917, steps: 5
768 (array([22]),) False
accuracy:  2.5666666666666664
Episode 768: reward: 4.418, steps: 5
769 (array([48]),) False
accuracy:  4.183333333333334
Episode 769: reward: 4.776, steps: 5
770 (array([22]),)

accuracy:  1.95
Episode 852: reward: 5.000, steps: 5
853 (array([22]),) False
accuracy:  1.7
Episode 853: reward: 5.000, steps: 5
854 (array([22]),) False
accuracy:  2.2333333333333334
Episode 854: reward: 5.000, steps: 5
855 (array([22]),) False
accuracy:  2.2333333333333334
Episode 855: reward: 3.763, steps: 5
856 (array([48]),) False
accuracy:  0.9166666666666666
Episode 856: reward: 4.658, steps: 5
857 (array([22]),) False
accuracy:  0.8333333333333333
Episode 857: reward: 3.769, steps: 5
858 (array([12]),) False
accuracy:  2.5833333333333335
Episode 858: reward: 5.000, steps: 5
859 (array([22]),) False
accuracy:  3.233333333333334
Episode 859: reward: 5.000, steps: 5
860 (array([12]),) False
accuracy:  3.0666666666666664
Episode 860: reward: 5.000, steps: 5
861 (array([22]),) False
accuracy:  1.8666666666666667
Episode 861: reward: 4.508, steps: 5
862 (array([14]),) False
accuracy:  0.8333333333333333
Episode 862: reward: 4.968, steps: 5


In [ ]:
avg_rwd = []
for rwd in cb_ep.rewards.values():
    if (len(rwd)!=0):
        avg_rwd.append(sum(rwd)/len(rwd))
plt.plot(avg_rwd)
#plt.xlabel("step")
#plt.ylabel("pos")

In [ ]:
avg_rwd = np.array(avg_rwd)
np.save('avg_rwd_{}.npy'.format(ENV_NAME), avg_rwd)

In [ ]:
import requests

def telegram(message):

    bot_token = '923260274:AAHyCqG6jpPI_xo0x2wJvhGVA_oeUnMf41Y'
    bot_chatID = '763781536'
    send = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + message
    try:
        response = requests.get(send)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)  
    return response.json()

message = "DQN done!\n"
telegram(message)